# Multilingual Knowledge Distillation

## Prepare data

In [1]:
import os

In [2]:
src = 'envi_data'
raw_data_dir = os.path.join(src, 'raw_dir')
raw_data_dir

'envi_data/raw_dir'

In [5]:
!ls

BANKING_77
create_test_data.ipynb
data
Embed-label_Setfit.ipynb
envi_data
Multilingual_knowledge_distill_train.py
Multi-lingual-training.ipynb
multi-phoMT
output.jpg
setfitckpt
setfitckpt_multilingual
setfitckpt_multilingual_distil
setfitckpt_multilingual_test_en
setfitckpt_multilingual_test_en_v1
setfitckpt_multilingual_test_en_v1_af
setfitckpt_multilingual_test_en_v1_af.tar
setfitckpt_multiphoMT
setfitckpt_multiphoMT_test
setfitckpt_onnx_multilingual_test_en
Setfit.ipynb
test_onnx.ipynb
train.py


In [4]:
if not os.path.exists(raw_data_dir):
    os.makedirs(raw_data_dir)

In [6]:
en_sens= []
with open('./envi_data/PhoMT/tokenization/train/train.en', 'r') as f:
    for line in f:
        en_sens.append(line.strip()) 

In [7]:
en_sens[20:50]

['He enriches " Midnight \'s Children " with a plethora of Indian and Pakistani cultural references , from family traditions to food , religion and folktales .',
 'Scribbling by night under the watchful eyes of his lover Padma , Saleem \'s frame narrative echoes that of " 1001 Nights , " where a woman named Scheherazade tells her king a series of stories to keep herself alive .',
 'And as Saleem sees it , 1001 is " the number of night , of magic , of alternative realities . "',
 'Over the course of the novel , Rushdie dazzles us with multiple versions of reality .',
 'Sometimes , this is like reading a rollercoaster .',
 'Saleem narrates : " Who what am I ?',
 'My answer : I am everyone everything whose being - in - the - world affected was affected by mine .',
 "I am anything that happens after I 've gone which would not have happened if I had not come .",
 "Nor am I particularly exceptional in this matter ; each ' I , ' every one of the now - six - hundred - million - plus of us , co

In [8]:
vi_sens = []
with open('./envi_data/PhoMT/tokenization/train/train.vi', 'r') as f:
    for line in f:
        vi_sens.append(line.strip()) 

In [9]:
vi_sens[20:50]

['" Những đứa con của nửa đêm " mang đậm màu sắc văn hoá của Ấn Độ và Pakistan , từ truyền thống gia đình , ẩm thực , đến tôn giáo hay văn hoá dân gian .',
 'Được viết nên mỗi đêm dưới sự chứng kiến của người tình là Padma , câu chuyện qua lời kể của Saleem mang âm hưởng nàng Scheherazade trong " Nghìn lẻ một đêm " , kể chuyện cho nhà vua hằng đêm để giữ mạng sống của mình .',
 'Và theo cách Saleem nhìn nhận , 1001 là " con số của đêm , của ma thuật , và hiện thực huyền ảo . "',
 'Xuyên suốt tiểu thuyết , Rushdie khiến người đọc choáng ngợp bởi những phiên bản khác nhau của hiện thực .',
 'Đôi khi , việc đọc cuốn sách này cũng như đi tàu lượn vậy .',
 'Saleem tự thuật : " Tôi là ai ? là gì ?',
 'Trả lời : Tôi là tổng hợp mọi thứ diễn ra , là tất cả mọi người , mọi vật mà sự tồn tại tác động đến tôi và bị tôi tác động .',
 'Tôi là bất cứ chuyện gì xảy ra khi tôi ra đi nhưng sẽ không xảy ra nếu tôi không xuất hiện .',
 'Tôi cũng không phải là ngoại lệ cá biệt trong vấn đề này ; mỗi một "

In [10]:
len(en_sens)

2977999

In [11]:
train_dict_data = []

for en_s, vi_s in zip(en_sens, vi_sens):
    train_dict_data.append(en_s+'\t'+vi_s)

In [12]:
train_dict_data[:10]

['It begins with a countdown .\tCâu chuyện bắt đầu với buổi lễ đếm ngược .',
 'On August 14th , 1947 , a woman in Bombay goes into labor as the clock ticks towards midnight .\tNgày 14 , tháng 8 , năm 1947 , gần nửa đêm , ở Bombay , có một phụ nữ sắp lâm bồn .',
 'Across India , people hold their breath for the declaration of independence after nearly two centuries of British occupation and rule .\tCùng lúc , trên khắp đất Ấn , người ta nín thở chờ đợi tuyên ngôn độc lập sau gần hai thập kỷ là thuộc địa của Anh .',
 'And at the stroke of midnight , a squirming infant and two new nations are born in perfect synchronicity .\tKhi đồng hồ điểm thời khắc nửa đêm , một đứa trẻ và hai quốc gia mới , cùng một lúc , ra đời .',
 'These events form the foundation of " Midnight \'s Children , " a dazzling novel by the British - Indian author Salman Rushdie .\tNhững sự kiện này là nền móng tạo nên " Những đứa con của nửa đêm , " một tiểu thuyết nổi tiếng của nhà văn Anh gốc Ấn Salman Rushdie .',
 "T

In [13]:
len(train_dict_data)

2977999

In [33]:
import gzip

with gzip.open(f'./envi_data/phoMT-train.tsv.gz', 'wt', encoding = 'utf-8') as f:
    f.write('\n'.join(train_dict_data))

## Model

### Teacher model

In [14]:
from sentence_transformers import SentenceTransformer

/data2/nghiatl/anaconda3/envs/onnx/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [36]:
from sentence_transformers import models

In [38]:
teacher = SentenceTransformer('all-mpnet-base-v2')
teacher

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [45]:
mpnet = teacher[0]
pooler = teacher[1]

teacher = SentenceTransformer(modules=[mpnet, pooler])
teacher

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

## Student model

In [46]:
student = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
student

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

## Fine-tuning

In [47]:
from sentence_transformers import ParallelSentencesDataset

data = ParallelSentencesDataset(student_model=student, teacher_model=teacher, batch_size=32, use_embedding_cache=True)

In [48]:
max_sentences_per_language = 1000000
train_max_sentence_length = 250 # max num of characters per sentence

data.load_data('./envi_data/phoMT-train.tsv.gz', max_sentences=max_sentences_per_language, max_sentence_length=train_max_sentence_length)

In [49]:
from torch.utils.data import DataLoader

loader = DataLoader(data, shuffle=True, batch_size=32)

In [50]:
from sentence_transformers import losses

loss = losses.MSELoss(model=student)

In [52]:
from sentence_transformers import evaluation
import numpy as np

epochs = 1
warmup_steps = int(len(loader) * epochs * 0.1)

student.fit(
    train_objectives=[(loader, loss)],
    epochs=epochs,
    warmup_steps=warmup_steps,
    output_path='./multi-phoMT',
    optimizer_params={'lr': 2e-5, 'eps': 1e-6},
    save_best_model=True,
    show_progress_bar=False
)

/data2/nghiatl/anaconda3/envs/SEO/lib/python3.8/site-packages/sentence_transformers/SentenceTransformer.py:547: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  labels = torch.tensor(labels)
